In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch.amp import autocast, GradScaler
import wandb

wandb.login(key="d6f8c99f1fd73267470842bbf00f03ae845f7308")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m014 (da24m014-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/kaggle/input/nature/nature_12K/inaturalist_12K"
batch_size = 64
img_size = 224
num_classes = 10
epochs = 10
lr = 1e-4


In [3]:
train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

In [4]:
full_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])
test_ds = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=batch_size, num_workers=2)

In [5]:
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
for param in model.features.parameters():
    param.requires_grad = False  # Freeze convolutional layers

# Replace classifier head
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 199MB/s] 


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
scaler = GradScaler()

In [7]:
wandb.init(project="DLA2-PartB", name="VGG16-finetune", config={
    "strategy": "freeze features, fine-tune classifier",
    "model": "VGG16",
    "epochs": epochs,
    "batch_size": batch_size,
    "lr": lr
})
wandb.watch(model)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [8]:
def train_one_epoch(loader):
    model.train()
    total_loss, correct = 0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Use autocast with device_type set to 'cuda' (for GPU) or 'cpu' (for CPU)
        with autocast(device_type="cuda" if torch.cuda.is_available() else "cpu"):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(loader.dataset), correct / len(loader.dataset)



def evaluate(loader):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Use autocast with device_type set to 'cuda' (for GPU) or 'cpu' (for CPU)
            with autocast(device_type="cuda" if torch.cuda.is_available() else "cpu"):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            total_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
    
    return total_loss / len(loader.dataset), correct / len(loader.dataset)


In [9]:
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(train_loader)
    val_loss, val_acc = evaluate(val_loader)
    print(f"[Epoch {epoch+1}] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")
    wandb.log({
        "epoch": epoch+1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

[Epoch 1] Train Acc: 0.4928 | Val Acc: 0.5675
[Epoch 2] Train Acc: 0.6067 | Val Acc: 0.6100
[Epoch 3] Train Acc: 0.6667 | Val Acc: 0.6170
[Epoch 4] Train Acc: 0.6962 | Val Acc: 0.6320
[Epoch 5] Train Acc: 0.7281 | Val Acc: 0.6360
[Epoch 6] Train Acc: 0.7565 | Val Acc: 0.6360
[Epoch 7] Train Acc: 0.7903 | Val Acc: 0.6415
[Epoch 8] Train Acc: 0.8066 | Val Acc: 0.6340
[Epoch 9] Train Acc: 0.8286 | Val Acc: 0.6210
[Epoch 10] Train Acc: 0.8435 | Val Acc: 0.6380


In [10]:
torch.save(model.state_dict(), "vgg16_finetuned.pth")
wandb.save("vgg16_finetuned.pth")

['/kaggle/working/wandb/run-20250418_112257-a39madzx/files/vgg16_finetuned.pth']

**RESNET**

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast, GradScaler
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import wandb

wandb.login(key="d6f8c99f1fd73267470842bbf00f03ae845f7308")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m014 (da24m014-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/kaggle/input/nature/nature_12K/inaturalist_12K"
batch_size = 64
img_size = 224
num_classes = 10
epochs = 10
lr = 1e-4

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

In [4]:
full_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])
test_ds = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=batch_size, num_workers=2)

# Load pre-trained ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 210MB/s]


In [6]:
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Unfreeze only the new fc layer
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
scaler = GradScaler()

In [9]:
wandb.init(project="DLA2-PartB", name="ResNet50-finetune", config={
    "strategy": "freeze all except final fc layer",
    "model": "ResNet50",
    "epochs": epochs,
    "batch_size": batch_size,
    "lr": lr
})
wandb.watch(model)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [10]:
def train_one_epoch(loader):
    model.train()
    total_loss, correct = 0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_type="cuda" if torch.cuda.is_available() else "cpu"):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item() * inputs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

def evaluate(loader):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast(device_type="cuda" if torch.cuda.is_available() else "cpu"):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [11]:
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(train_loader)
    val_loss, val_acc = evaluate(val_loader)
    print(f"[Epoch {epoch+1}] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")
    wandb.log({
        "epoch": epoch+1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

[Epoch 1] Train Acc: 0.6111 | Val Acc: 0.6330
[Epoch 2] Train Acc: 0.6537 | Val Acc: 0.6515
[Epoch 3] Train Acc: 0.6712 | Val Acc: 0.6605
[Epoch 4] Train Acc: 0.6868 | Val Acc: 0.6635
[Epoch 5] Train Acc: 0.6970 | Val Acc: 0.6750
[Epoch 6] Train Acc: 0.6967 | Val Acc: 0.6800
[Epoch 7] Train Acc: 0.7066 | Val Acc: 0.6920
[Epoch 8] Train Acc: 0.7086 | Val Acc: 0.6955
[Epoch 9] Train Acc: 0.7146 | Val Acc: 0.6960
[Epoch 10] Train Acc: 0.7087 | Val Acc: 0.6940


In [12]:
torch.save(model.state_dict(), "resnet50_finetuned.pth")
wandb.save("resnet50_finetuned.pth")

['/kaggle/working/wandb/run-20250418_140108-zbaztvkj/files/resnet50_finetuned.pth']

RESNET-2

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torch.amp import autocast, GradScaler
import wandb

wandb.login(key="d6f8c99f1fd73267470842bbf00f03ae845f7308")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m014 (da24m014-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/kaggle/input/nature/nature_12K/inaturalist_12K"
batch_size = 64
img_size = 224
num_classes = 10
epochs = 10
lr = 1e-4

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])
val_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

In [4]:
full_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])
test_ds = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=batch_size, num_workers=2)

model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 173MB/s] 


In [5]:
for param in model.parameters():
    param.requires_grad = False

# Unfreeze last ResNet block (layer4) and fc
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

# Replace the final classification layer
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
scaler = GradScaler()

In [6]:
wandb.init(project="DLA2-PartB", name="ResNet50-finetune-layer4+fc", config={
    "strategy": "unfreeze layer4 and fc",
    "model": "ResNet50",
    "epochs": epochs,
    "batch_size": batch_size,
    "lr": lr
})
wandb.watch(model)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [7]:
def train_one_epoch(loader):
    model.train()
    total_loss, correct = 0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_type="cuda" if torch.cuda.is_available() else "cpu"):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * inputs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(loader.dataset), correct / len(loader.dataset)

# Evaluation
def evaluate(loader):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast(device_type="cuda" if torch.cuda.is_available() else "cpu"):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [8]:
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(train_loader)
    val_loss, val_acc = evaluate(val_loader)
    print(f"[Epoch {epoch+1}] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")
    wandb.log({
        "epoch": epoch+1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

[Epoch 1] Train Acc: 0.6776 | Val Acc: 0.7415
[Epoch 2] Train Acc: 0.8155 | Val Acc: 0.7570
[Epoch 3] Train Acc: 0.8684 | Val Acc: 0.7645
[Epoch 4] Train Acc: 0.9082 | Val Acc: 0.7815
[Epoch 5] Train Acc: 0.9312 | Val Acc: 0.7735
[Epoch 6] Train Acc: 0.9485 | Val Acc: 0.7670
[Epoch 7] Train Acc: 0.9575 | Val Acc: 0.7700
[Epoch 8] Train Acc: 0.9660 | Val Acc: 0.7730
[Epoch 9] Train Acc: 0.9730 | Val Acc: 0.7790
[Epoch 10] Train Acc: 0.9767 | Val Acc: 0.7720


In [ ]:
torch.save(model.state_dict(), "resnet50_finetuned_layer4_fc.pth")
wandb.save("resnet50_finetuned_layer4_fc.pth")

RESNET-3

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch.amp import GradScaler
import wandb

wandb.login(key="d6f8c99f1fd73267470842bbf00f03ae845f7308")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m014 (da24m014-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/kaggle/input/nature/nature_12K/inaturalist_12K"
batch_size = 64
img_size = 224
num_classes = 10
epochs = 10
lr = 1e-4

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((img_size + 32, img_size + 32)),
    transforms.RandomCrop(img_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

In [4]:
full_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])
test_ds = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=batch_size, num_workers=2)

# Load pre-trained ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 221MB/s]


In [5]:
for param in model.parameters():
    param.requires_grad = False
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

# Modify classifier for 10-class classification
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
scaler = GradScaler()

In [6]:
wandb.init(project="DLA2-PartB", name="ResNet50-finetune-augmented", config={
    "strategy": "unfreeze layer4 and fc",
    "model": "ResNet50",
    "epochs": epochs,
    "batch_size": batch_size,
    "lr": lr,
    "augmentation": True
})
wandb.watch(model)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [7]:
def train_one_epoch(loader):
    model.train()
    total_loss, correct = 0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_type="cuda"):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * inputs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(loader.dataset), correct / len(loader.dataset)

def evaluate(loader):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast(device_type="cuda"):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [8]:
def test(loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            correct += (outputs.argmax(1) == labels).sum().item()
    return correct / len(loader.dataset)

In [9]:
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(train_loader)
    val_loss, val_acc = evaluate(val_loader)
    print(f"[Epoch {epoch+1}] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")
    wandb.log({
        "epoch": epoch+1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

[Epoch 1] Train Acc: 0.6133 | Val Acc: 0.7080
[Epoch 2] Train Acc: 0.7283 | Val Acc: 0.7185
[Epoch 3] Train Acc: 0.7726 | Val Acc: 0.7295
[Epoch 4] Train Acc: 0.7898 | Val Acc: 0.7370
[Epoch 5] Train Acc: 0.8106 | Val Acc: 0.7375
[Epoch 6] Train Acc: 0.8337 | Val Acc: 0.7480
[Epoch 7] Train Acc: 0.8454 | Val Acc: 0.7545
[Epoch 8] Train Acc: 0.8670 | Val Acc: 0.7600
[Epoch 9] Train Acc: 0.8789 | Val Acc: 0.7460
[Epoch 10] Train Acc: 0.8886 | Val Acc: 0.7345


In [10]:
test_acc = test(test_loader)
print(f"Test Accuracy: {test_acc:.4f}")
wandb.log({"test_acc": test_acc})

torch.save(model.state_dict(), "resnet50_finetuned.pth")
wandb.save("resnet50_finetuned.pth")

Test Accuracy: 0.7615


['/kaggle/working/wandb/run-20250418_163616-rkunqgf5/files/resnet50_finetuned.pth']